In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [3]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from '/Users/huynh/Desktop/Python/EnSF/AllenCahn/AllenCahn_Solver.py'>

In [57]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
T = 10
Nt = 1000
eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

S = 2

cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

# U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [59]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

In [61]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(obs, x0, time_steps, C, score_likelihood=None, drift_fun=f, \
                diffuse_fun=g, alpha_fun=cond_alpha, sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = np.random.randn(x0.shape[0], x0.shape[1])
    t = 1.0
    
    path_all = []
    t_vec = []
    
    # define storage
    if save_path:
        path_all.append(xt.copy())
        t_vec.append(t)
    
    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood(xt, t, obs, C)) +np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
        
#         xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt.copy())
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [63]:
ndim = Nx*Ny
spa_indices = np.random.permutation(ndim)[:int(1* ndim)] #100 observations

# spa_indices = np.random.permutation(ndim)[:int(0.7* ndim)] #100 observations
idx_obs = np.sort(spa_indices)

# If needed, run "AllenCahn_Solver_DegenerateMob" for a new reference solution

In [65]:
data1 = scipy.io.loadmat('RefSol_AllenC_T10_DegenerateMob_max0_v1.mat')
U_Ref = data1['RefU']

URef = U_Ref.T

In [67]:
# ntEnSF = 500
ntEnSF = 250
t0 = 0
filtering_steps = ntEnSF
timeTrue = np.linspace(0, 1, Nt+1)
tEnSF = np.linspace(0, 1, filtering_steps+1)
indices_time = np.searchsorted(timeTrue, tEnSF, side='left')

state_timeextract = URef[indices_time, :].copy()

U_EnSF = state_timeextract[:, idx_obs].copy()

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1
SDE_Sigma = 0.01

eps_alpha = 0.05

# ensemble size
ensemble_size = 60
ensemble_true = 1
# forward Euler step
euler_steps = 500
def g_tau(t):
    return 1-t

U0_EnSF = np.random.uniform(-0.9, 0.9, size=(ensemble_size, Nx*Ny))

rmse_all = []
obs_save = []
est_save = np.zeros((filtering_steps+1, ndim))
est_save[[0], :] += np.mean(U0_EnSF, axis=0)

In [49]:
def score_likelihood(xt, t, obs, C):
    # obs: (d)
    # xt: (ensemble, d)
    score_x = -(np.arctan(xt) - obs)/obs_sigma**2 * (1./(1+xt**2))
    tau = g_tau(t)
    return tau * score_x / C

In [69]:
U1_EnSF = np.zeros((ensemble_size, Nx*Ny))
noise = 0.4*np.random.randn(Nx*Ny) # for 2N(0, I)
# noise = 0.1*np.random.randn(Nx*Ny) # for N(0, I)
for ll in range(ensemble_size):
    U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0_EnSF[ll, :], N, dtEnSF, eps, gamma, S, Dh, 4, noise)
    U1_EnSF[ll, :] += U1
    
    
noiseU = np.sqrt(dtEnSF) * SDE_Sigma * np.random.randn(*U1_EnSF.shape)
U1_EnSF += noiseU

In [70]:
state_scale = U_EnSF[[1], :].copy()
    
indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                          ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]

indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                          ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]

indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                          ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                          ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                          ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                          ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]

indob_scale6 = np.nonzero(((-1e-7 <= state_scale) & (state_scale < -1e-8)) |
                          ((1e-8 <= state_scale) & (state_scale < 1e-7)))[1]

indob_scale7 = np.nonzero(((-1e-8 <= state_scale) & (state_scale < -1e-9)) |
                          ((1e-9 <= state_scale) & (state_scale < 1e-8)))[1]

indob_scale8 = np.nonzero(((-1e-9 <= state_scale) & (state_scale < -1e-10)) |
                          ((1e-10 <= state_scale) & (state_scale < 1e-9)))[1]

indob_scale9 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                          ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]

indob_scale10 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                          ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]

indob_scale11 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                          ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]

indob_scale12 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                          ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]

indob_scale13 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                          ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
indob_scale14 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                          ((0 <= state_scale) & (state_scale < 1e-17)))[1]

state_scale[:, indob_scale0] *= 1e1
state_scale[:, indob_scale1] *= 1e2
state_scale[:, indob_scale2] *= 1e3
state_scale[:, indob_scale3] *= 1e4
state_scale[:, indob_scale4] *= 1e5
state_scale[:, indob_scale5] *= 1e6
state_scale[:, indob_scale6] *= 1e7
state_scale[:, indob_scale7] *= 1e8
state_scale[:, indob_scale8] *= 1e9
state_scale[:, indob_scale9] *= 1e12
state_scale[:, indob_scale10] *= 1e13
state_scale[:, indob_scale11] *= 1e14
state_scale[:, indob_scale12] *= 1e15
state_scale[:, indob_scale13] *= 1e16
state_scale[:, indob_scale14] *= 1e17

obs = np.arctan(state_scale.copy())
obs += np.random.randn(*U_EnSF[[1], :].shape) * obs_sigma

x0_EnSF = U1_EnSF[:, idx_obs].copy()
for l in range(5):      
    indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
    indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
    indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
    indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
    indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
    indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
    indx_scale6 = np.argwhere(((-1e-7<=x0_EnSF) & (x0_EnSF<-1e-8)) | ((1e-8<=x0_EnSF) & (x0_EnSF<1e-7)))
    indx_scale7 = np.argwhere(((-1e-8<=x0_EnSF) & (x0_EnSF<-1e-9)) | ((1e-9<=x0_EnSF) & (x0_EnSF<1e-8)))
    indx_scale8 = np.argwhere(((-1e-9<=x0_EnSF) & (x0_EnSF<-1e-10)) | ((1e-10<=x0_EnSF) & (x0_EnSF<1e-9)))
    indx_scale9 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
    indx_scale10 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
    indx_scale11 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
    indx_scale12 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
    indx_scale13 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
    indx_scale14 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))

    x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
    x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
    x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
    x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
    x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
    x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
    x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e7
    x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e8
    x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e9
    x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e12
    x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e13
    x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e14
    x0_EnSF[indx_scale12[:, 0], indx_scale12[:, 1]] *= 1e15
    x0_EnSF[indx_scale13[:, 0], indx_scale13[:, 1]] *= 1e16
    x0_EnSF[indx_scale14[:, 0], indx_scale14[:, 1]] *= 1e17

    sln_bar = reverse_SDE(obs, x0=x0_EnSF.copy(), time_steps=euler_steps,  C=1, score_likelihood=score_likelihood)

    sln_bar[:, indob_scale0] /= 1e1
    sln_bar[:, indob_scale1] /= 1e2
    sln_bar[:, indob_scale2] /= 1e3
    sln_bar[:, indob_scale3] /= 1e4
    sln_bar[:, indob_scale4] /= 1e5
    sln_bar[:, indob_scale5] /= 1e6
    sln_bar[:, indob_scale6] /= 1e7
    sln_bar[:, indob_scale7] /= 1e8
    sln_bar[:, indob_scale8] /= 1e9
    sln_bar[:, indob_scale9] /= 1e12
    sln_bar[:, indob_scale10] /= 1e13
    sln_bar[:, indob_scale11] /= 1e14
    sln_bar[:, indob_scale12] /= 1e15
    sln_bar[:, indob_scale13] /= 1e16
    sln_bar[:, indob_scale14] /= 1e17

    x0_EnSF = sln_bar.copy()
        
U1_EnSF[:, idx_obs] = x0_EnSF.copy()
est_save[[1], :] += np.mean(U1_EnSF, axis=0)

In [73]:
for i in range(1, filtering_steps):
    print(f'step={i}:')
    t1 = time.time()    
    
    state_scale = U_EnSF[[i+1], :].copy()
    
    indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                              ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]
    
    indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                              ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]
    
    indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                              ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

    indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                              ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

    indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                              ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
    indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                              ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]
    
    indob_scale6 = np.nonzero(((-1e-7 <= state_scale) & (state_scale < -1e-8)) |
                              ((1e-8 <= state_scale) & (state_scale < 1e-7)))[1]

    indob_scale7 = np.nonzero(((-1e-8 <= state_scale) & (state_scale < -1e-9)) |
                              ((1e-9 <= state_scale) & (state_scale < 1e-8)))[1]
    
    indob_scale8 = np.nonzero(((-1e-9 <= state_scale) & (state_scale < -1e-10)) |
                              ((1e-10 <= state_scale) & (state_scale < 1e-9)))[1]
    
    indob_scale9 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                              ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]
    
    indob_scale10 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                              ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]
    
    indob_scale11 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                              ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]
    
    indob_scale12 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                              ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]
    
    indob_scale13 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                              ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
    indob_scale14 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                              ((0 <= state_scale) & (state_scale < 1e-17)))[1]
    
    state_scale[:, indob_scale0] *= 1e1
    state_scale[:, indob_scale1] *= 1e2
    state_scale[:, indob_scale2] *= 1e3
    state_scale[:, indob_scale3] *= 1e4
    state_scale[:, indob_scale4] *= 1e5
    state_scale[:, indob_scale5] *= 1e6
    state_scale[:, indob_scale6] *= 1e7
    state_scale[:, indob_scale7] *= 1e8
    state_scale[:, indob_scale8] *= 1e9
    state_scale[:, indob_scale9] *= 1e12
    state_scale[:, indob_scale10] *= 1e13
    state_scale[:, indob_scale11] *= 1e14
    state_scale[:, indob_scale12] *= 1e15
    state_scale[:, indob_scale13] *= 1e16
    state_scale[:, indob_scale14] *= 1e17
    
    obs = np.arctan(state_scale.copy())
    obs += np.random.randn(*U_EnSF[[i+1], :].shape) * obs_sigma
       
    x_state = np.zeros((ensemble_size, ndim))
    noise = 0.4*np.random.randn(Nx*Ny)
    # noise = 0.1*np.random.randn(Nx*Ny) # for N(0, I)
    for ll in range(ensemble_size):
        x_state[ll, :] = AC_Neu.AllenCahn_Solver_1step_BDF2(U1_EnSF[ll, :], U0_EnSF[ll, :], N, dtEnSF,\
                                                            eps, gamma, S, Dh, b0, b1, 4, noise)
    
    
    noiseU = np.sqrt(dtEnSF) * SDE_Sigma * np.random.randn(*x_state.shape)
    
    x_state += noiseU
    
    x0_EnSF = x_state[:, idx_obs].copy()

    for l in range(5):      
        indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
        indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
        indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
        indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
        indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
        indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
        indx_scale6 = np.argwhere(((-1e-7<=x0_EnSF) & (x0_EnSF<-1e-8)) | ((1e-8<=x0_EnSF) & (x0_EnSF<1e-7)))
        indx_scale7 = np.argwhere(((-1e-8<=x0_EnSF) & (x0_EnSF<-1e-9)) | ((1e-9<=x0_EnSF) & (x0_EnSF<1e-8)))
        indx_scale8 = np.argwhere(((-1e-9<=x0_EnSF) & (x0_EnSF<-1e-10)) | ((1e-10<=x0_EnSF) & (x0_EnSF<1e-9)))
        indx_scale9 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
        indx_scale10 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
        indx_scale11 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
        indx_scale12 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
        indx_scale13 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
        indx_scale14 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))
    
        x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
        x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
        x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
        x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
        x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
        x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
        x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e7
        x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e8
        x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e9
        x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e12
        x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e13
        x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e14
        x0_EnSF[indx_scale12[:, 0], indx_scale12[:, 1]] *= 1e15
        x0_EnSF[indx_scale13[:, 0], indx_scale13[:, 1]] *= 1e16
        x0_EnSF[indx_scale14[:, 0], indx_scale14[:, 1]] *= 1e17
    
        sln_bar = reverse_SDE(obs, x0=x0_EnSF.copy(), time_steps=euler_steps,  C=1, score_likelihood=score_likelihood)

        sln_bar[:, indob_scale0] /= 1e1
        sln_bar[:, indob_scale1] /= 1e2
        sln_bar[:, indob_scale2] /= 1e3
        sln_bar[:, indob_scale3] /= 1e4
        sln_bar[:, indob_scale4] /= 1e5
        sln_bar[:, indob_scale5] /= 1e6
        sln_bar[:, indob_scale6] /= 1e7
        sln_bar[:, indob_scale7] /= 1e8
        sln_bar[:, indob_scale8] /= 1e9
        sln_bar[:, indob_scale9] /= 1e12
        sln_bar[:, indob_scale10] /= 1e13
        sln_bar[:, indob_scale11] /= 1e14
        sln_bar[:, indob_scale12] /= 1e15
        sln_bar[:, indob_scale13] /= 1e16
        sln_bar[:, indob_scale14] /= 1e17

        # print(sln_bar[:, Size_U+Size_V+np.arange(0, Size_P)])
        x0_EnSF = sln_bar.copy()
     
    x_state[:, idx_obs] = sln_bar.copy()
    x_state = np.clip(x_state, -1, 1)
    
    U0_EnSF = U1_EnSF.copy()
    U1_EnSF = x_state.copy()
    # print(q_batch)
    est_save[[i+1], :] += np.mean(x_state, axis=0)
#     print(est_save[i+1, -1])
    # get rmse
    rmse_temp = np.sqrt(np.mean((est_save[[i+1], :] - state_timeextract[[i+1], :])**2))

    # get time
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2 - t1:.4f}')

    # save information
    rmse_all.append(rmse_temp)

    # check divergence
    if rmse_temp > 1000:
        print('diverge!')
        break

step=1:
	 RMSE = 0.2587
	 time = 33.4024
step=2:
	 RMSE = 0.2024
	 time = 33.2100
step=3:
	 RMSE = 0.1724
	 time = 33.0581
step=4:
	 RMSE = 0.1541
	 time = 33.4634
step=5:
	 RMSE = 0.1414
	 time = 33.3804
step=6:
	 RMSE = 0.1333
	 time = 33.1729
step=7:
	 RMSE = 0.1275
	 time = 33.1174
step=8:
	 RMSE = 0.1237
	 time = 32.8225
step=9:
	 RMSE = 0.1207
	 time = 33.0152
step=10:
	 RMSE = 0.1189
	 time = 32.4822
step=11:
	 RMSE = 0.1179
	 time = 33.0694
step=12:
	 RMSE = 0.1165
	 time = 33.0292
step=13:
	 RMSE = 0.1163
	 time = 33.0742
step=14:
	 RMSE = 0.1164
	 time = 32.1253
step=15:
	 RMSE = 0.1162
	 time = 32.3395
step=16:
	 RMSE = 0.1149
	 time = 32.1531
step=17:
	 RMSE = 0.1147
	 time = 32.2328
step=18:
	 RMSE = 0.1157
	 time = 32.2514
step=19:
	 RMSE = 0.1154
	 time = 32.4115
step=20:
	 RMSE = 0.1150
	 time = 32.7650
step=21:
	 RMSE = 0.1150
	 time = 32.7801
step=22:
	 RMSE = 0.1144
	 time = 32.7838
step=23:
	 RMSE = 0.1147
	 time = 32.9621
step=24:
	 RMSE = 0.1147
	 time = 32.3834
s

In [75]:
scipy.io.savemat('ResultEnSF_AllenC_DegMob_T10_max0_Nt250_2N0I_100Obs_v1.mat', {'U_EnSF':est_save, 'rmse': rmse_all})

# scipy.io.savemat('ResultEnSF_AllenC_DegMob_T10_max0_Nt250_2N0I_70Obs_v1.mat', {'U_EnSF':est_save, 'rmse': rmse_all})